# Atommodell

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook werden wir analog zum ersten Teil des Jupyter-Notebooks [5-02-Harmonischer-Oszillator.ipynb](5-02-Harmonischer-Oszillator.ipynb) über den harmonischen Oszillator vorgehen. Allerdings werden wir jetzt ein Potential betrachten, für das die Eigenenergien und die dazugehörigen Eigenzustände nicht analytisch berechnet werden können. Das Potential 

$$ V(x) = - \frac{\alpha^3}{\sqrt{\alpha^2+x^2}} $$

soll dabei ein eindimensionales Modellpotential für ein Atom darstellen. Im Gegensatz zum harmonischen Oszillator wird es hier auch möglich sein, ungebundene Zustände zu berechnen. Allerdings muss man dabei bedenken, dass das verwendete Ortsgitter das Atommodell effektiv in einen unendlich tiefen Potentialtopf einbettet.

## Importanweisungen

In [ ]:
import numpy as np
import numpy.linalg as LA
import ipywidgets as widgets
from ipywidgets import interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Berechnung des Potentials

Durch die Verwendung der Wurzelfunktion aus dem NumPy-Paket kann die Funktion `potential` neben skalaren Werten für das Argument `x` auch NumPy-Arrays verarbeiten.

In [ ]:
def potential(x, alpha):
    return -alpha**3/np.sqrt(alpha**2+x**2)

## Berechnung des Hamilton-Operators

Diese Funktion ist identisch mit der Funktion `hamilton_operator` aus dem Jupyter-Notebook [5-02-Harmonischer-Oszillator.ipynb](5-02-Harmonischer-Oszillator.ipynb).

In [ ]:
def hamilton_operator(n_max, x_max, alpha):
    ndim = 2*n_max+1
    x, dx = np.linspace(-x_max, x_max, ndim, retstep=True)
    h = (np.eye(ndim)
         - 0.5*(np.eye(ndim, k=1) + np.eye(ndim, k=-1)))
    h = h/dx**2 + np.diag(potential(x, alpha))
    return h

## Berechnung der Eigenwerte und Eigenvektoren

Diese Funktion ist identisch mit der Funktion `eigenproblem` aus dem Jupyter-Notebook [5-02-Harmonischer-Oszillator.ipynb](5-02-Harmonischer-Oszillator.ipynb).

In [ ]:
def eigenproblem(n_max, x_max, alpha):
    h = hamilton_operator(n_max, x_max, alpha)
    evals, evecs = LA.eigh(h)
    return evals, evecs

## Implementierung der Bedienelemente und graphische Darstellung ausgewählter Eigenzustände

Mit den Schiebereglern lassen sich die folgenden Parameter einstellen:
- `n_max`: Anzahl der Gitterpunkte auf einer Seite des Ursprungs. Insgesamt werden `2*n_max+1` Gitterpunkte verwendet.
- `x_max`: Ausdehnung des Gitters von `-x_max` bis `x_max`
- `x_plot_max`: Ausdehnung des darzustellenden Bereichs von `-x_plot_max` bis `x_plot_max`
- `alpha`: Potentialparameter

Es werden die fünf energetisch niedrigsten Eigenzustände sowie der vierzigste angeregte Eigenzustand dargestellt, wobei die Nulllinie um die jeweilige Eigenenergie verschoben ist. Zur besseren Interpretation der Resultate wird des Potential ebenfalls eingezeichnet.

In [ ]:
widget_dict = {"n_max":
               widgets.IntSlider(
                   value=1000, min=50, max=1000, step=10,
                   description=r"$n_\text{max}$"),
               "x_max":
               widgets.FloatSlider(
                   value=80, min=10, max=100, step=1,
                   description=r"$x_\text{max}$"),
               "x_plot_max":
               widgets.FloatSlider(
                   value=20, min=1, max=100, step=1,
                   description=r"$x_\text{plot, max}$"),
               "alpha":
               widgets.FloatSlider(
                   value=2, min=0.5, max=10, step=0.1,
                   description=r"$\alpha$")
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_result(n_max, x_max, x_plot_max, alpha):
    evals, evecs = eigenproblem(n_max, x_max, alpha)
    x_values = np.linspace(-x_max, x_max, 2*n_max+1)

    fig, ax = plt.subplots()
    for n in range(5):
        ax.plot(x_values, evecs[:, n]+evals[n])
    ax.plot(x_values, 4*evecs[:, 40]+evals[40])
    ax.plot(x_values, potential(x_values, alpha),
            color="black")
    ax.set_xlabel("$x$")
    ax.set_ylabel(r"$\psi(x)$")
    ax.set_xlim((-x_plot_max, x_plot_max))